In [ ]:
import torch
import numpy as np
from model import MLP
from test import test_model
from utils import *
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from dataset import simpleDataset, uniformDataset, contextUniformDataset
import os 

cuda = torch.cuda.is_available()
num_workers = 4 if cuda else 0 
device = torch.device("cuda" if cuda else "cpu")
verbose = True

In [ ]:
class MLP24(nn.Module):
    def __init__(self, size_list, activation="relu", batch_norm=True):
        super(MLP24, self).__init__()
        layers = []
        self.size_list = size_list
        if batch_norm:
            layers.append(nn.BatchNorm1d(num_features=size_list[0]))
        for i in range(len(size_list) - 2):
            layers.append(nn.Linear(size_list[i],size_list[i+1]))
            if batch_norm:
                layers.append(nn.BatchNorm1d(num_features=size_list[i+1]))
            if activation is "relu":
                layers.append(nn.ReLU())
            else:
                raise NotImplementedError
        layers.append(nn.Linear(size_list[-2], size_list[-1]))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [ ]:
# model 24
# hyper parameters
orig_x_dim, orig_y_dim = 40, 138
context_size = 16
input_size, output_size = orig_x_dim * (2*context_size+1), orig_y_dim
size_list = [input_size, 1024, 512, 512, 256, output_size]
activation = "relu" # use only one type of activation
lr = 1e-3 # default lr is 1e-3
epochs = 20
batch_size = 512

# model
print("building model...")
model24 = MLP24(size_list, activation)
criterion24 = nn.CrossEntropyLoss()
optimizer24 = optim.Adam(model24.to(device).parameters(), lr=1e-3)

# check if load checkpoint
if os.path.exists("../checkpoint/checkpoint_24.tar"):
    print("checkpoint file ../checkpoint/checkpoint_24.tar exist.")
    checkpoint24 = load_checkpoint("../checkpoint/checkpoint_24.tar", 
                                 model24, optimizer24)

print_model_statistics(checkpoint24)

In [ ]:
# model 27
# hyper parameters
orig_x_dim, orig_y_dim = 40, 138
context_size = 16
input_size, output_size = orig_x_dim * (2*context_size+1), orig_y_dim
size_list = [input_size, 512, 512, 512, 256, output_size]
activation = "relu" # use only one type of activation
lr = 1e-3 # default lr is 1e-3
epochs = 15
batch_size = 512

# model
print("building model...")
model27 = MLP(size_list, activation)
criterion27 = nn.CrossEntropyLoss()
optimizer27 = optim.Adam(model27.to(device).parameters(), lr=1e-3)

# check if load checkpoint
if os.path.exists("../checkpoint/checkpoint_27.tar"):
    print("checkpoint file ../checkpoint/checkpoint_27.tar exist.")
    checkpoint27 = load_checkpoint("../checkpoint/checkpoint_27.tar", 
                                 model27, optimizer27)

print_model_statistics(checkpoint27)

In [ ]:
data_path = "../data"
path_testx = "%s/test.npy" % data_path
testx = np.load(path_testx, allow_pickle=True)
test_dataset = contextUniformDataset(testx, is_test=True, context_size=context_size)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    num_workers=num_workers
)

In [ ]:
predicts = torch.LongTensor().to(device)
    
with torch.no_grad():
    model24.eval()
    model27.eval()

    model24.to(device)
    model27.to(device)

    # no target in test dataset/data loader
    for batch_idx, data in enumerate(test_loader):
        data = data.to(device)

        outputs24 = model24(data)
        outputs27 = model27(data)

        output = outputs24 + outputs27

        _, predict = torch.max(output.data, 1)

        predicts = torch.cat([predicts, predict])

assert predicts.shape[0] == len(test_loader.dataset)

result = np.concatenate([np.arange(len(predicts)).reshape(-1, 1),
                         predicts.detach().cpu().clone().numpy().reshape(-1, 1)], axis=1)
np.savetxt("../data/test_pred_2427.csv", result, fmt="%i", delimiter=",", header="id,label", comments="")
